# Laboratorium 4 - Odległość edycyjna, LCS

## Odległość edycyjna

In [2]:
from enum import Enum

class Operation(Enum):
    add = 0
    remove = 1
    sub = 2
    leave = 3

def edit_distance(s1, s2):
    l1, l2 = len(s1) + 1, len(s2) + 1
    
    operations = [[]] * l1
    edit = [[]] * l1
    for i in range(l1):
        edit[i] = [0] * l2
        operations[i] = [(0, 0, -1, -1)] * l2

    for j in range(l2):
        edit[0][j] = j
        operations[0][j] = (Operation.add, (0, j - 1), -1, -1)
    for i in range(l1):
        edit[i][0] = i
        operations[i][0] = (Operation.remove, i - 1, -1, -1)

    for i in range(1, l1):
        for j in range(1, l2):
            m = edit[i - 1][j] + 1
            op = (Operation.remove, i - 1, i - 1, j)

            if edit[i][j - 1] + 1 < m:
                m = edit[i][j - 1] + 1
                op = (Operation.add, (i, j - 1), i, j - 1)

            match = 0 if s1[i - 1] == s2[j - 1] else 1
            if edit[i - 1][j - 1] + match < m:
                m = edit[i - 1][j - 1] + match
                if match == 0:
                    op = (Operation.leave, i - 1, i - 1, j - 1)
                else:
                    op = (Operation.sub, (i - 1, j - 1), i - 1, j - 1)
        
            edit[i][j] = m
            operations[i][j] = op

    op_seq = []
    i, j = l1 - 1, l2 - 1
    while i >= 0 and j >= 0:
        if (operations[i][j][0] != Operation.leave 
            and not ((operations[i][j][0] == Operation.remove or operations[i][j][0] == Operation.add) and operations[i][j][1] == -1)):
            op_seq.append((operations[i][j][0], operations[i][j][1]))
        i, j = operations[i][j][2], operations[i][j][3]

    return edit[l1 - 1][l2 - 1], op_seq

In [3]:
def print_result(t1, t2):
    dist, op_seq = edit_distance(t1, t2)
    print(f"Odległość edycyjna: {dist}")
    
    for op in op_seq:
        if op[0] == Operation.sub:
            print(f"Zamiana {t1[op[1][0]]} -> {t2[op[1][1]]}")
        elif op[0] == Operation.add:
            print(f"Dodanie {t2[op[1][1]]}")
        elif op[0] == Operation.remove:
            print(f"Usunięcie {t1[op[1]]}")

In [4]:
print_result("los", "kloc")

Odległość edycyjna: 2
Zamiana s -> c
Dodanie k


In [5]:
print_result("Łódź", "Lodz")

Odległość edycyjna: 3
Zamiana ź -> z
Zamiana ó -> o
Zamiana Ł -> L


In [6]:
print_result("kwintesencja", "quintessence")

Odległość edycyjna: 5
Usunięcie a
Zamiana j -> e
Dodanie s
Zamiana w -> u
Zamiana k -> q


In [7]:
print_result("ATGAATCTTACCGCCTCG", "ATGAGGCTCTGGCCCCTG")

Odległość edycyjna: 7
Usunięcie C
Usunięcie G
Dodanie G
Zamiana A -> G
Dodanie C
Zamiana T -> G
Zamiana A -> G


## Najdłuższy wspólny podciąg

In [8]:
def longest_common_subsequence(x, y):
    m, n = len(x) + 1, len(y) + 1
    C = [[]] * m
    for i in range(m):
        C[i] = [0] * n

    for i in range(1, m):
        for j in range(1, n):
            if x[i - 1] == y[j - 1]:
                C[i][j] = C[i - 1][j - 1] + 1
            else:
                C[i][j] = max(C[i][j - 1], C[i - 1][j])
    
    i, j = m - 1, n - 1
    sigma = []

    while i > 0 and j > 0:
        if C[i][j] == C[i - 1][j - 1] + (1 if x[i - 1] == y[j - 1] else 0):
            if x[i - 1] == y[j - 1]:
                sigma.insert(0, x[i - 1])
            i = i - 1
            j = j - 1
        elif C[i][j] == C[i - 1][j]:
            i = i - 1
        else:
            j = j - 1

    return C[m - 1][n - 1], sigma

In [9]:
lcs, _ = longest_common_subsequence("ATGAATCTTACCGCCTCG", "ATGAGGCTCTGGCCCCTG")
print(lcs)

13


Aby dokonać tokenizacji wykorzystałem bibliotekę spaCy. Wygenerowałem plik *romeo-i-julia-2.txt* (znajduje się w archiwum)

In [10]:
from spacy.tokenizer import Tokenizer
from spacy.lang.pl import Polish
import random
from lcs import longest_common_subsequence

def tokenize(file):
    nlp = Polish()
    tokenizer = Tokenizer(nlp.vocab)

    with open(file, 'r') as content_file:
        content = content_file.read()
        tokens = tokenizer(content)
        return [token for token in tokens]

def remove_tokens(tokens, frac):
    random.seed()
    to_remove = int(frac * len(tokens))
    new_tokens = list(tokens)
    for _ in range(to_remove):
        i = random.randint(0, len(new_tokens) - 1)
        del new_tokens[i]

    return new_tokens

In [13]:
tokens = tokenize('romeo-i-julia-700.txt')
smaller_set = remove_tokens(tokens, 0.3)
lines = str(' '.join(map(lambda token: token.text, smaller_set))).split('\n')

print("Tokens: " + str(len(tokens)) + ", smaller set: " + str(len(smaller_set)))

Tokens: 2272, smaller set: 1591


In [16]:
l, sigma = longest_common_subsequence(tokens, smaller_set)
print(f"Longest common subsequence between files: {l}")

Longest common subsequence between files: 1591


## Narzędzie diff

In [32]:
import sys

def read_file(file):
    with open(file, 'r') as content_file:
        return content_file.read()

def diff(file1, file2):
    lines1 = read_file(file1).split('\n')
    lines2 = read_file(file2).split('\n')
    
    l, sigma = longest_common_subsequence(lines1, lines2)
    output = []

    def print_line(line, c, i):
        if line not in sigma:
            output.append(f"{c} ({i}) {line}")

    i, j = 0, 0
    while i < len(lines1) or j < len(lines2):
        if i < len(lines1):
            print_line(lines1[i], '>', i)
            i = i + 1
        if j < len(lines2):
            print_line(lines2[j], '<', j)
            j = j + 1
    
    return output

# Wykomentowane w notebooku
            
# if __name__ == "__main__":
#     args = list(sys.argv)

#     if len(args) != 3:
#         print("Usage: python diff.py file1 file2")
#         exit(1)

#     output = diff(args[1], args[2])
#     print('\n'.join(output))

In [31]:
output = diff("romeo-i-julia-700.txt", "romeo-i-julia-2.txt")

# Wyświetlam w notebooku tylko 20 pierwszych linii
print('\n'.join(output[0:20]))

> (0) William Shakespeare
< (0) William Romeo i Julia Józef
> (2) Romeo i Julia
< (2) ISBN
> (3) tłum. Józef Paszkowski
> (5) ISBN 978-83-288-2903-9
< (6) ESKALUS — książę panujący — młody krewny księcia
< (7) MONTEKI, KAPULET dwóch nieprzyjaznych sobie * STARZEC — stryjeczny brat Kapuleta ROMEO — syn * MERKUCJO — krewny księcia Montekiego
< (8) * — krewny Kapulet
> (9) OSOBY:
< (9) * LAURENTY — ojciec
> (10)  * ESKALUS — książę panujący w Weronie
< (10) * JAN brat z zgromadzenia * — służący Romea
> (11)  * PARYS — młody Weroneńczyk szlachetnego rodu, krewny księcia
< (11) * SAMSON, słudzy
> (12)  * MONTEKI, KAPULET — naczelnicy dwóch domów nieprzyjaznych sobie
< (12) * ABRAHAM — służący Montekiego
> (13)  * STARZEC — stryjeczny brat Kapuleta
< (13) * APTEKARZ
> (14)  * ROMEO — syn Montekiego
